## Imports

In [6]:
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import pandas as pd
import xlsxwriter

## Selenium Setup

In [7]:
def openChrome():
    portz = 9995
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument(f'--remote-debugging-port={portz}')
    chromeOptions.add_experimental_option("excludeSwitches", ["enable-automation"])
    chromeOptions.add_argument("--disable-blink-features=AutomationControlled")
    chromeOptions.add_argument("--disable-blink-features")
    prefs = {"profile.managed_default_content_settings.images": 2}
    chromeOptions.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(executable_path=r"C:/Users/kylewang/Desktop/MLSA_2024/Upwork/LoopNet/drivers/chromedriver", options=chromeOptions)
    driver.set_window_rect(0, 0, width=360, height=600)
    driver.set_page_load_timeout(66)
    return driver

## Scraper Code

In [8]:
# On the new tab that opens - login to RCA
# Move over to the holdings tab and pull up the table view (3 lines)
# Hit the enter key on the code below and the scrape will start

In [9]:
driver = openChrome()
list_write = []

url =  f'https://app.rcanalytics.com/home'
driver.get(url)
sleep(1)

driver.set_window_rect(0, 0, width=1222, height=999)

output_file_path = 'rca_holdings.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    pass

/var/folders/2p/mkvb6kbd08l30n0fx72jcwz80000gn/T/ipykernel_45646/2566852624.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:/Users/kylewang/Desktop/MLSA_2024/Upwork/LoopNet/drivers/chromedriver", options=chromeOptions)


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [13]:
counter = 0
while 1:
    page_source  = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    data = soup.find_all("tr",attrs={"ng-click": "pinHoldingsTableController.onRowSelected(row)"})
    
    for item in data:
        counter += 1
        print('------------------------------------------')
        
        # Box Tab 1
        property_name, addr1, addr2 = "", "", ""
        address = item.find("td", class_="address")
        address = address.find_all("span", attrs={"tooltip-placement": "top-left"})
        try:
            property_name = address[0].text
        except:
            property_name = ""
        try:
            addr1 = address[1].text
        except:
            addr1 = ""
        try:
            addr2 = address[2].text
        except:
            addr2 = ""       

        # Box Tab 2
        units, year_built, floors = "", "", ""
        propertyInfo = item.find("td", class_="propertyInfo")
        propertyInfo = propertyInfo.find_all("span", attrs={"tooltip-placement": "top-left"})
        try:
            units = propertyInfo[0].text
        except:
            units = ""
        try:
            year_built = propertyInfo[1].text
        except:
            year_built = ""
        try:
            floors = propertyInfo[2].text
        except:
            floors = ""

        # Box Tab 3
        property_type, subtype, feature = "", "", ""
        propertyType = item.find("td", class_="propertyType")
        propertyType = propertyType.find_all("span", attrs={"tooltip-placement": "top-left"})
        try:
            property_type = propertyType[0].text
        except:
            property_type = ""
        try:
            subtype = propertyType[1].text
        except:
            subtype = ""
        try:
            feature = propertyType[2].text
        except:
            feature = ""

        # Box Tab 4
        owner = ""
        owner_ = item.find("td", class_="owner")
        owner_ = owner_.find_all("span", attrs={"tooltip-placement": "top-left"})
        try:
            owner = owner_[0].text
        except:
            owner = ""

        # Box Tab 5
        last_trans_type, last_trans_date, last_trans_price = "", "", ""
        lastTransaction = item.find("td", class_="lastTransaction")
        lastTransaction = lastTransaction.find_all("span", attrs={"tooltip-placement": "top-left"})
        try:
            last_trans_type = lastTransaction[0].text
        except:
            last_trans_type = ""
        try:
            last_trans_date = lastTransaction[1].text
        except:
            last_trans_date = ""
        try:
            last_trans_price = lastTransaction[2].text
        except:
            last_trans_price = ""

        # Box Tab 6
        loan_amount, lender_name = "", ""
        loanAndLender = item.find("td", class_="loanAndLender")
        loanAndLender = loanAndLender.find_all("span", attrs={"tooltip-placement": "top-left"})
        try:
            loan_amount = loanAndLender[0].text
        except:
            loan_amount = ""
        try:
            lender_name = loanAndLender[1].text
        except:
            lender_name = ""

        # Box Tab 7
        tenant = ""
        tenants_ = item.find("td", class_="tenants")
        tenants_ = tenants_.find_all("span", attrs={"tooltip-placement": "top-left"})
        try:
            tenant = info[0].text
        except:
            tenant = ""

        # Compile to excel row
        list_write = []
        list_write.append([property_name,
                          addr1,
                          addr2,
                          units,
                          year_built,
                          floors,
                          property_type,
                          subtype,
                          feature,
                          owner,
                          last_trans_type,
                          last_trans_date,
                          last_trans_price,
                          loan_amount,
                          lender_name,
                          tenant])
        print(list_write)
        print(counter)
        
        rows = list(zip(*list_write))
        cols_names = ['Property_Name', 
                     'Address_1',
                     'Address_2',
                     'Units',
                     'Year_Built',
                     'Floors',
                     'Property_Type',
                     'Subtype',
                     'Feature',
                     'Owner',
                     'Last_Trans_Type',
                     'Last_Trans_Date',
                     'Last_Trans_Price',
                     'Loan_Amount',
                     'Lender_Name',
                     'Tenant']
        df_new_rows = pd.DataFrame(dict(zip(cols_names, rows)))

        df_existing = pd.read_excel(output_file_path)
        df_combined = pd.concat([df_existing, df_new_rows], ignore_index=True)
        df_combined.to_excel(output_file_path, index=False)

    kkk = 1
    while 1:
        try:
            element = WebDriverWait(driver,0.1).until(
                EC.presence_of_element_located((By.XPATH, f"(//tr[@ng-click='pinHoldingsTableController.onRowSelected(row)'])[{kkk}]"))
            )
        except:
            driver.execute_script("arguments[0].scrollIntoView();", element) 
            break
        kkk += 1

------------------------------------------
[['super8temple ', '5505 S General Bruce Dr ', 'Temple, TX 76502  USA', '94 units', ' ', '', 'Hotel ', 'Limited Service ', '', '', ' ', ' ', ' ', '', '', '']]
------------------------------------------
[['stayAPT Suites Hotel ', '3225 Galeno Crossing ', 'Brownsville, TX 78526  USA', '50 units', '2023 ', '1 bldg/2 flrs', 'Hotel ', 'Limited Service ', 'extended stay', 'stayAPT Suites', 'Construction ', "Oct '21 ", 'n/a ', '', '', '']]
------------------------------------------
[['stayAPT Suites ', '3051 Plaza Centre Ct ', 'Bryan, TX 77802  USA', '70 units', '2023 ', '1 bldg/3 flrs', 'Hotel ', 'Limited Service ', 'suite', 'stayAPT Suites', 'Construction ', "Jul '22 ", 'n/a ', '', '', '']]
------------------------------------------
[['sleep inn and suites ', 'W 10th Street ', 'Fort Stockton, TX 79735  USA', '49 units', ' ', '', 'Hotel ', 'Limited Service ', '', '', ' ', ' ', ' ', '', '', '']]
------------------------------------------
[['quality i

------------------------------------------
[['aLoft Austin ', '11601 Domain Dr ', 'Austin, TX 78758  USA', '140 units', '2009 ', '6 flrs', 'Hotel ', 'Limited Service ', '', 'LodgeWorks', 'Refinance ', "Jul '17 ", 'n/a ', '$20.5 ', 'Emprise Bank', '']]
------------------------------------------
[['Z Resort ', '2517 Anderson Gibson Rd ', 'Grapevine, TX 76051  USA', '1,376 units', ' ', '', 'Hotel ', 'Full-Service ', '', '', ' ', ' ', ' ', '', '', '']]
------------------------------------------
[['Yonder Inn Beeville ', '2201 N Frontage Rd ', 'Beeville, TX 78102  USA', '79 units', '2014 ', '1 bldg/3 flrs', 'Hotel ', 'Limited Service ', '', 'Alphahotels Corp', 'Sale ', "Apr '21 ", 'n/a ', '$2.1 ', 'Wallis State Bank', '']]
------------------------------------------
[['Ye Kendall Inn ', '128 W Blanco Rd ', 'Boerne, TX 78006  USA', '36 units', '1859/2013 ', '2 flrs', 'Hotel ', 'Limited Service ', '', 'Bird Properties, LLC', 'Sale ', "Jun '15 ", 'n/a ', '', '', '']]
---------------------------

------------------------------------------
[['Woodspring Suites Lubbock West ', '2311 W Loop 289 ', 'Lubbock, TX 79407  USA', '104 units', '2005/2016 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service ', 'extended stay', 'Blackstone JV Starwood Capital', 'Sale ', "Feb '22 ", "$8.6 alloc'd", '$7.0 ', 'BXSC 2022-WSS', '']]
------------------------------------------
[['Woodspring Suites Laredo ', '1103 Bob Bullock Loop ', 'Laredo, TX 78043  USA', '121 units', '2007/2016 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service ', 'suite', 'Blackstone JV Starwood Capital', 'Sale ', "Feb '22 ", "$10.0 alloc'd", '$8.2 ', 'BXSC 2022-WSS', '']]
------------------------------------------
[['Woodspring Suites Houston ', '5350 W Sam Houston Pkwy N ', 'Houston, TX 77041  USA', '122 units', '2017 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service ', 'suite', 'Four VP Texas LP', 'Construction ', "Sep '16 ", 'n/a ', '$5.6 ', 'Alliance Bank', '']]
------------------------------------------
[['Woodspring Suites Hotel ', 

------------------------------------------
[['Woodspring Suites ', '5210 183A Toll Rd ', 'Leander, TX 78641  USA', '122 units', '2021 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service ', 'extended stay', 'BRR Architecture', 'Construction ', "May '20 ", 'n/a ', '', '', '']]
------------------------------------------
[['Woodspring Suites ', '5210 183A Toll Rd ', 'Cedar Park, TX 78613  USA', '122 units', '2021 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service ', 'extended stay', 'WoodSpring Hotels', 'Construction ', "Mar '20 ", 'n/a ', '', '', '']]
------------------------------------------
[['Woodspring Suites ', '1207 W Expy 83 ', 'Pharr, TX 78577  USA', '121 units', '2006/2016 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service ', 'extended stay', 'T3 Capital LP', 'Sale ', "Aug '20 ", 'n/a ', '$7.2 ', '', '']]
------------------------------------------
[['Woodspring Suites ', '1201 Louis Henna Blvd ', 'Round Rock, TX 78664  USA', '122 units', '2021 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service '

------------------------------------------
[['WoodSpring Suites Baytown ', '4624 East Fwy ', 'Baytown, TX 77521  USA', '124 units', '2008 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service ', 'suite', 'WSS Houston LP', 'Sale ', "Sep '18 ", 'n/a ', '$17.1 ', 'Alliance Bank', '']]
------------------------------------------
[['WoodSpring Suites Austin Round Rock ', '1950 N I35 ', 'Round Rock, TX 78681  USA', '121 units', '2007/2016 ', '4 flrs', 'Hotel ', 'Limited Service ', 'extended stay', 'Starwood Capital ; Blackstone', 'Sale ', "Feb '22 ", '$14.6 approx', '$12.0 ', 'BXSC 2022-WSS', '']]
------------------------------------------
[['WoodSpring Suites Abilene ', '4266 Oil Belt Ln ', 'Abilene, TX 79605  USA', '116 units', '2010 ', '4 flrs', 'Hotel ', 'Limited Service ', 'suite', 'RS Baypoint Row LLC; Warner Row LLC', 'Sale ', "Aug '22 ", 'n/a ', '', '', '']]
------------------------------------------
[['WoodSpring Suites ', '6801 K Ave ', 'Plano, TX 75025  USA', '122 units', '2019 ', '1 bldg

...

------------------------------------------
[['La Quinta Inn ', '13290 Fm 1960 Rd W ', 'Houston, TX 77065  USA', '130 units', '1986/1999 ', '1 bldg/3 flrs', 'Hotel ', 'Limited Service ', '', 'Manish Patel', 'Sale ', "Jan '20 ", 'n/a ', '$2.3 ', 'PCB Bank', '']]
------------------------------------------
[['La Quinta Inn ', '5922 I-10 W ', 'San Antonio, TX 78201  USA', '111 units', '1973/2002 ', '3 bldgs/2 flrs', 'Hotel ', 'Limited Service ', '', 'Jaid Hospitality Development LLC', 'Sale ', "Jan '20 ", 'n/a ', '$3.3 ', 'Wallis State Bank', '']]
------------------------------------------
[['La Quinta Inn ', '1310 Marketplace Dr ', 'Garland, TX 75041  USA', '65 units', '2017 ', '1 bldg/3 flrs', 'Hotel ', 'Limited Service ', '', 'Aid Inc', 'Sale ', "Jun '20 ", '$4.2 approx', '$3.3 ', 'Bank of Hope', '']]
------------------------------------------
[['La Quinta Inn ', '431 Airport Fwy ', 'Euless, TX 76040  USA', '71 units', '2009 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service ', '', 'Mushtaq 

------------------------------------------
[['La Quinta Del Sol ', '9595 John W Elliott Dr ', 'Frisco, TX 75034  USA', '101 units', '2019 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service ', '', 'Twilight Hospitality LP, Twilight Hotels LLC', 'Construction ', "Feb '19 ", 'n/a ', '$8.3 ', 'First United Bank', '']]
------------------------------------------
[['La Quinta Del Sol ', '2550 State 121 Hwy ', 'Euless, TX 76039  USA', '85 units', '2019 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service ', 'suite', 'Amish Patel', 'Construction ', "Mar '18 ", 'n/a ', '', '', '']]
------------------------------------------
[['La Quinta Dallas Grand Prairie ', '380 E Palace Pkwy ', 'Grand Prairie, TX 75050  USA', '85 units', '2016 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service ', 'suite', 'Ramanlal Patel', 'Refinance ', "Feb '18 ", 'n/a apprsl', '$6.6 ', 'CSAIL 2018-CX11', '']]
------------------------------------------
[['La Quinta - Wichita Falls ', '2511 E Montgomery Pl ', 'Wichita Falls, TX 76308  USA

------------------------------------------
[['La Quinta - Inn & Suites -- Houston (Galleria Area ', '1625 W Loop S ', 'Houston, TX 77027  USA', '173 units', '1997/1998 ', '1 bldg/4 flrs', 'Hotel ', 'Limited Service ', '', 'Five Rivers Hospitality LLC', 'Sale ', "Feb '22 ", "$10.0 confm'd", '', '', '']]
------------------------------------------
[['La Quinta - Dallas Mesquite ', '118 US-80 ', 'Mesquite, TX 75149  USA', '60 units', '2001 ', '1 bldg/3 flrs', 'Hotel ', 'Limited Service ', 'suite', 'Makanji LLC', 'Sale ', "Nov '21 ", 'n/a ', '$4.0 ', 'Golden Bank', '']]
------------------------------------------
[['La Quinta - Dallas Fort Worth, TX ', '1809 Hwy 121 ', 'Bedford, TX 76021  USA', '76 units', '2008 ', '', 'Hotel ', 'Limited Service ', '', 'Roshan Patel; Jayanti Patel', 'Refinance ', "Dec '14 ", 'n/a apprsl', '$5.3 ', 'CWCapital Asset Mgmt OBO MSBAM 2015-C20', '']]
------------------------------------------
[['La Quinta - Dallas ', '123 Lodging Ave ', 'Coppell, TX 75019  USA', '

------------------------------------------
[['La Copa Hotel ', '2000 S 10th St ', 'McAllen, TX 78503  USA', '108 units', '1950 ', '2 flrs', 'Hotel ', 'Limited Service ', '', 'VRJ DPB LLC', 'Sale ', "Aug '14 ", 'n/a ', '', '', '']]
------------------------------------------
[['La Cantera Resort & Spa ', '16641 La Cantera Pkwy ', 'San Antonio, TX 78256  USA', '496 units', '1996/2016 ', '7 bldgs/4 flrs', 'Hotel ', 'Full-Service ', 'suite', 'Ohana Real Estate Investors', 'Sale ', "Jun '21 ", 'n/a ', '$188.4 ', 'Blackstone', '']]
------------------------------------------
[['LAURA LODGE MOTEL & SUITES ', '1000 East Business 20 ', 'Pecos, TX 79772  USA', '28 units', '1970 ', '2 flrs', 'Hotel ', 'Limited Service ', '', '', ' ', ' ', ' ', '', '', '']]
------------------------------------------
[['LA Quinta Inn & Suites Luling ', '203 E Hwy 90 ', 'Luling, TX 78648  USA', '55 units', '2015 ', '1 bldg/3 flrs', 'Hotel ', 'Limited Service ', 'suite', 'Tushar Kabaria', 'Sale ', "Dec '20 ", 'n/a ', '

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=121.0.6167.160)
Stacktrace:
0   chromedriver                        0x0000000104b73168 chromedriver + 4673896
1   chromedriver                        0x0000000104b6a9c3 chromedriver + 4639171
2   chromedriver                        0x000000010475efdd chromedriver + 397277
3   chromedriver                        0x0000000104764443 chromedriver + 418883
4   chromedriver                        0x0000000104766a97 chromedriver + 428695
5   chromedriver                        0x00000001047ed6e6 chromedriver + 980710
6   chromedriver                        0x00000001047cd8c2 chromedriver + 850114
7   chromedriver                        0x00000001047ec796 chromedriver + 976790
8   chromedriver                        0x00000001047cd663 chromedriver + 849507
9   chromedriver                        0x000000010479d1cf chromedriver + 651727
10  chromedriver                        0x0000